In [4]:
from sentence_transformers import SentenceTransformer

/workspaces/llm-zoomcamp/03-vector-search/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

In [6]:
user_question = "I just discovered the course. Can I still join it?"

In [7]:
# Q1
encoding = embedding_model.encode(user_question)
encoding[0]

0.078222655

In [8]:
# Q2 Load the Documents with IDs

import requests
from tqdm.auto import tqdm

docs_url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/documents-with-ids.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for document in tqdm(documents_raw):
    if document['course'] == 'machine-learning-zoomcamp':
        documents.append(document)
    

  0%|          | 0/948 [00:00<?, ?it/s]

In [9]:
len(documents)

375

In [10]:
# Q2 create embeddings
embeddings = []

for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

  0%|          | 0/375 [00:00<?, ?it/s]

In [11]:
import numpy as np

X = np.array(embeddings)
X.shape

(375, 768)

In [12]:
# Q3 search
v = encoding
scores = X.dot(v)
scores.max()

0.6506573

In [13]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]


In [14]:
search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [15]:
# Q4 evaluate with Hit-rate
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [18]:
## calculate relevance
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    v_doc_question = embedding_model.encode(q['question'])
    results = search_engine.search(v_doc_question, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [19]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [20]:
hit_rate(relevance_total)

0.9398907103825137

In [22]:
# Q5 indexing with elastic search
len(v)

768

In [42]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://127.0.0.1:9200', request_timeout=500) 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "document": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': False, 'index': 'course-questions'})

In [34]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [36]:
# add fields for embeddings to be saved
for doc in tqdm(ground_truth):
    question = doc['question']
    
    doc['question_vector'] = embedding_model.encode(question)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [45]:
es_client.cluster.health()

ObjectApiResponse({'cluster_name': 'docker-cluster', 'status': 'red', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 0, 'active_shards': 0, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 2, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 0.0})

In [46]:
es_client.indices.get_settings(index=index_name)

ObjectApiResponse({'course-questions': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'course-questions', 'creation_date': '1720989834391', 'number_of_replicas': '0', 'uuid': 'lrPJJIybRqKOS5eg83OOPw', 'version': {'created': '8040399'}}}}})

In [44]:
# index all documents with all fields
for doc in tqdm(ground_truth):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/1830 [00:00<?, ?it/s]

ApiError: ApiError(503, 'unavailable_shards_exception', '[course-questions][0] primary shard is not active Timeout: [1m], request: [BulkShardRequest [[course-questions][0]] containing [index {[course-questions][HqIAs5AB4Z5JY0XUTK59], source[n/a, actual length: [15.9kb], max length: 2kb]}]]')